# A Knowledge Graph for the SEEKCommons Project

The Resource Hub is a *[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph)* curated by the __[SEEKCommons project](https://seekcommons.org/)__ that connects the various resources and entities in the SEEKCommons project. The graph is built using the [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) database and is accessible via the [SPARQL](https://en.wikipedia.org/wiki/SPARQL)query language.



## Where to start?

The graph is centered around the concept of a *resource*. A resource can be anything from a researcher, a research organization, a software tool, a dataset or a publication. Each resource is connected to other resources through various relationships. For example, a publication can be connected to a researcher who created it, a project that funded it, a publication that used it, or a software tool that processed it.

### Researchers

In [20]:
query = '''
SELECT ?researcher ?orcid ?name
WHERE
{
  # instance of (https://www.wikidata.org/wiki/Property:P31) = human (https://www.wikidata.org/wiki/Q5)
  ?researcher wdt:P31  wd:Q5 .
  # occupation (https://www.wikidata.org/wiki/Property:P106) = researcher (https://www.wikidata.org/wiki/Q1650915)
  ?researcher wdt:P106 wd:Q1650915 .
  # ORCID iD (https://www.wikidata.org/wiki/Property:P496) => ?orcid
  ?researcher wdt:P496 ?orcid .
  # given name (https://www.wikidata.org/wiki/Property:P735) => ?given_name
  # name in native language (https://www.wikidata.org/wiki/Property:P1559) => ?name
  ?researcher wdt:P1559 ?name .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 100
'''

In [21]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
df.head(10)

,researcher,name,orcid
0,http://www.wikidata.org/entity/Q40116614,Burak Yılmaz,0000-0002-7101-363X
1,http://www.wikidata.org/entity/Q92341,Hans Ulrich Gumbrecht,0000-0001-6364-7723
2,http://www.wikidata.org/entity/Q105927,根岸 英一,0000-0002-5746-5779
3,http://www.wikidata.org/entity/Q106949,Saul Perlmutter,0000-0002-4436-4661
4,http://www.wikidata.org/entity/Q192112,Peter Ware Higgs,0000-0002-5587-5379
5,http://www.wikidata.org/entity/Q90203912,Piotr Nowak,0000-0001-8631-5705
6,http://www.wikidata.org/entity/Q265310,مصطفى السيد,0000-0002-7674-8424
7,http://www.wikidata.org/entity/Q64682395,Barbara Zabłocka,0000-0003-3116-7295
8,http://www.wikidata.org/entity/Q450572,Jacques Laskar,0000-0003-2634-789X
9,http://www.wikidata.org/entity/Q57442733,Katarzyna Sawicka,0000-0002-6182-2994
